In [151]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import desc,row_number,when,col,max
from pyspark.sql.window import Window 

In [152]:
spark = SparkSession.builder.master('local').appName('Player with Longest Streak').getOrCreate()

In [153]:
df_input = spark.read.options(header='True', InferSchema='True').csv('players_results.csv').orderBy('player_id','match_date')
#df_input.show()

In [157]:
##filtering out consecutive winning streak each time for each player
df_input.createOrReplaceTempView('t')
df_output = spark.sql('select distinct player_id,match_result, count(*) as consecutive_count from (select player_id,match_result, (row_number() over (order by player_id) - row_number() over (partition by match_result order by player_id)) as grp from t  ) t  group by player_id,grp, match_result')
df_output = df_output.filter((col("match_result") == 'W'))
#df_output.show()

In [158]:
##finding out the max consecutive winning streak for each players
df_output = df_output.groupBy('player_id').max('consecutive_count').select(col('player_id'),col('max(consecutive_count)').alias('winning_streak'))
#df_output.show()

In [159]:
##finding out max streak players
df_output.createOrReplaceTempView('fb')
df_output = spark.sql(' select player_id,winning_streak from ( select player_id,winning_streak, dense_rank() over( order by winning_streak desc) as rnk from fb) a where a.rnk =1');
df_output.show()

+---------+--------------+
|player_id|winning_streak|
+---------+--------------+
|      402|             5|
|      403|             5|
+---------+--------------+

